In [1]:
import numpy as np
import pandas as pd
#remove warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
acled_data = pd.read_csv("1900-01-01-2022-05-18-PRIO-GRID-ID.csv", index_col=0)[['year','PRIO GRID ID']]
acled_data['Number of ACLED Events'] = 1
print(acled_data.shape)
acled_data

(1430515, 3)


,year,PRIO GRID ID,Number of ACLED Events
0,2022,176111,1
1,2022,175391,1
2,2022,175391,1
3,2022,175391,1
4,2022,176111,1
...,...,...,...
1430510,1997,141456,1
1430511,1997,141456,1
1430512,1997,111994,1
1430513,1997,109835,1


In [18]:
acled_data.describe()

,year,PRIO GRID ID,Number of ACLED Events
count,1.427500e+06,1.427500e+06,1427500.0
mean,2.018165e+03,1.625143e+05,1.0
std,3.903099e+00,2.789529e+04,0.0
min,1.997000e+03,5.062400e+04,1.0
25%,2.017000e+03,1.474390e+05,1.0
50%,2.019000e+03,1.690150e+05,1.0
75%,2.021000e+03,1.811550e+05,1.0
max,2.022000e+03,2.342050e+05,1.0


In [3]:
acled_data.drop(acled_data[acled_data['PRIO GRID ID'] == "No lat/longs"].index, inplace=True)

In [4]:
acled_data['PRIO GRID ID'] = acled_data['PRIO GRID ID'].astype(int)

In [5]:
year_df = pd.DataFrame({'year':range(1990,2023)})
prio_grid_df = pd.DataFrame({'PRIO GRID ID':acled_data['PRIO GRID ID'].unique()})
year_df['key'] = 1
prio_grid_df['key'] = 1
base_df = pd.merge(year_df, prio_grid_df, on ='key').drop("key", 1)

In [6]:
acled_data_grouped = acled_data.groupby(['year','PRIO GRID ID']).sum().reset_index()

In [7]:
acled_data_grouped = acled_data_grouped.sort_values(by=['PRIO GRID ID','year'], ascending=True).reset_index(drop=True)
acled_data_grouped = acled_data_grouped[['PRIO GRID ID','year','Number of ACLED Events']]

In [8]:
acled_data_grouped

,PRIO GRID ID,year,Number of ACLED Events
0,50624,2018,4
1,50624,2019,2
2,50624,2020,7
3,50624,2021,5
4,52065,2018,2
...,...,...,...
81817,231887,2020,1
81818,231887,2021,1
81819,232042,2021,4
81820,232345,2020,1


In [9]:
new_df = pd.merge(base_df, acled_data_grouped, on = ['year','PRIO GRID ID'], how = 'left').fillna(int(0))
new_df = new_df.sort_values(by=['PRIO GRID ID','year'], ascending=True).reset_index(drop=True)

In [12]:
print(new_df.shape)

(649374, 3)


In [16]:
new_df = new_df[['PRIO GRID ID','year','Number of ACLED Events']]
new_df.to_csv('acled_data_grouped.csv',index=False)

ALternative Code (Lot of Time Required)

In [ ]:
# Equate Number of ACLED Events to 0 for years where there are no events from 1997 to 2022
for i, prio_grid in enumerate(acled_data_grouped['PRIO GRID ID'].unique()):
    if(i%1000==0):
        print(i)
    for year in range(1990,2022):
        if year not in acled_data_grouped[acled_data_grouped['PRIO GRID ID']==prio_grid]['year'].unique():
            acled_data_grouped = acled_data_grouped.append({'PRIO GRID ID':prio_grid,'year':year,'Number of ACLED Events':0},ignore_index=True)

In [ ]:
acled_data_grouped = acled_data_grouped.sort_values(by=['PRIO GRID ID','year'], ascending=True).reset_index(drop=True)